# **How to run rust + cuda code using google colab**

Google Colab gives you free access to T4 GPUs that you can use to run your CUDA projects. However, by default, it doesn't support rust or CUDA.

Before executing the following cells, ensure you are connected to a T4 instance (Check the top right icon).

![T4 icon](https://github.com/gkigiermo/rust_cuda_colab/blob/main/imgs/connectToHost.png)

You only need a Google account to proceed. If you are new to Google Colab, you need to press shift + enter to run the code in the cells (the grey cells). When executed correctly a green check is seen at the left and the output is displayed right below the cell.

If for any reason the instance loses connection or is not working, you might need to restart it. You can do that from the top menu `Runtime-> Restart runtime`.
Think of the instance as a VM, so if you restart it all the changes you did (installing tools or files generated) are erased and you need to execute them again.

The cells in this notebook are meant to be executed sequentially to fulfill dependencies. You can choose either to start by running the Rust or CUDA sections independently.

## **Mounting Google Drive**
Google Colab files are not stored unless you have a paid account.
To not losing changes, you should mount your Google Drive and work in a directory from there. This will also allow you to modify files locally and sync them with Google Colab easily.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

The above cell mounts your Google Drive in the ```/content/drive``` directory. You can check that it was mounted correctly by clicking on the folder icon at the left of your screen.

So far, It is not possible to mount only one specific directory, so be careful when running commands that might erase data.

I have previously created a folder named *workspace* at the root of my Google Drive. For security, I want to perform all the work there.
The next cell allows me to change the directory in the same way I would do it in a terminal. I use the `pwd` command to print the path and verify it.


In [ ]:
%cd /content/drive/MyDrive/workspace/
%pwd

## **Installing Rust**

By default, rust is not included in the VMs of Google Colab so it is necessary to install it. You need to do this step every time you access the Colab or if for any reason you have to restart the runtime. It is annoying, but it takes only about 15s.

In [ ]:
!apt install rustc cargo

You can check the version of the tools in the VM running the following cell.

In [ ]:
!rustc --version
!cargo --version
!nvcc --version
!gcc --version
!cmake --version

## **Cloning icicle and running the tests**

You can now proceed to clone the icicle repo. Remember that it will be stored in the workspace folder of your Google Drive. Note that you only need to clone it once, and then just keep it updated. The easiest way to perform the repo operations is to do it locally in your synced directory.

In [ ]:
!git clone https://github.com/ingonyama-zk/icicle.git

Compiling the repo takes around 4 minutes in the T4 instances, so be patient :P

In [ ]:
!ls -la
%cd icicle/icicle
!mkdir -p build
!cmake -S . -B build
!time cmake --build build

You might observe some warnings during compilation, but no fatal errors.
You can check if the library was compiled correctly by running the tests. This takes around 4m15s in the T4 instances.

In [ ]:
%cd build
!time ctest

All tests should pass.
If you arrived here, you are ready to go! Good luck and have fun!

# **Bonus: develop CUDA in google colab**

If you are interested in developing CUDA using Google Colab, you need to add a plugin. Note that rust-related cells are not a requirement to run/develop CUDA, so if you want you can just execute the steps from here. However, If you want to save your data you need to mount the Google Drive or copy them manually.

To change directory to the `/content` folder and verify it worked run:

In [ ]:
%cd /content/
%pwd

Install the nvcc4jupyter plugin

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Load the nvcc extension

In [ ]:
%load_ext nvcc_plugin

Let's test it with a simple code to check the current GPU.
Note the mark `%%cu` on top of the code, that means that when you press shift + enter the cell will compile CUDA code and execute it.

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>

int main() {
    int devId = -1;
    cudaGetDevice(&devId);

    int numDevs = 0;
    cudaGetDeviceCount(&numDevs);

    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);

    printf("Num devices %d\n", numDevs);
    printf("Device Id: %d\n", devId);
    printf("  Device name: %s\n", prop.name);
    printf("  Compute capability: %d.%d\n", prop.major, prop.minor);
    printf("  Memory available  (GB): %ld\n", prop.totalGlobalMem/(1024*1024*1024));
    printf("  Peak Memory Bandwidth (GB/s): %.2f\n\n",
          2.0*prop.memoryClockRate*(prop.memoryBusWidth/8)/1.0e6);

    return 0;
}

You can also store the file by using the mark `%%cuda` with the `--name` parameter.

In [ ]:
%%cuda --name test.cu
#include <stdio.h>
#include <stdlib.h>

int main() {
    int devId = -1;
    cudaGetDevice(&devId);

    int numDevs = 0;
    cudaGetDeviceCount(&numDevs);

    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);

    printf("Num devices %d\n", numDevs);
    printf("Device Id: %d\n", devId);
    printf("  Device name: %s\n", prop.name);
    printf("  Compute capability: %d.%d\n", prop.major, prop.minor);
    printf("  Memory available  (GB): %ld\n", prop.totalGlobalMem/(1024*1024*1024));
    printf("  Peak Memory Bandwidth (GB/s): %.2f\n\n",
          2.0*prop.memoryClockRate*(prop.memoryBusWidth/8)/1.0e6);

    return 0;
}

The file will be stored in a src folder created by the plugin. Enter in the directory and check that exists.

In [ ]:
%cd src/
%ls

You can also compile the file in the same way you would do it in your local terminal.

In [ ]:
!nvcc test.cu -o test

Check that the `test` executable was created, and execute it.

In [ ]:
!./test

Finally you can profile the executable using `nvprof`.

In [ ]:
!nvprof ./test

Now, you are ready to go! Good luck!